In [ ]:
import kagglehub
import pandas as pd
import ast

# Download latest version
path = kagglehub.dataset_download("dennisho/blackjack-hands")

print("Path to dataset files:", path)
path += '/blackjack_simulator.csv';

df = pd.read_csv(path, nrows=100000)

df['player_final'] = df['player_final'].apply(ast.literal_eval)
df['player_final'] = df['player_final'].explode('player_final')
#'target': df['win'][0:10000].map({'-1': 0, < 0: 1}).values.tolist()

Path to dataset files: /root/.cache/kagglehub/datasets/dennisho/blackjack-hands/versions/1


In [ ]:
# Map 'win' column: negative values to 0, 0 and above to 1
df['target'] = df['win'].apply(lambda x: 0 if x < 0 else 1) # thanks genimi again

print(df['target'])

0        1
1        1
2        1
3        1
4        0
        ..
99995    1
99996    1
99997    1
99998    1
99999    1
Name: target, Length: 100000, dtype: int64


In [ ]:
#df_first_elements = df['player_final'].apply(lambda col: [item[0] for item in col])
#df['player_final'] = df['player_final'].apply(lambda x: [item[0] for item in col])
df = df[df['player_final'].apply(lambda x: isinstance(x, list) and len(x) == 3 and all(isinstance(i, (int, float)) for i in x))]

#print(df['player_final'][df.index[0]])
print(df)


       shoe_id  cards_remaining  dealer_up initial_hand   dealer_final  \
1            0              411         10       [5, 5]        [10, 8]   
4            0              395          8      [6, 10]     [8, 2, 10]   
5            0              389          7       [3, 4]     [7, 2, 10]   
6            0              383          6       [3, 3]    [6, 10, 10]   
7            0              374          3      [10, 8]     [3, 10, 7]   
...        ...              ...        ...          ...            ...   
99987     1645              362         11      [6, 11]     [11, 5, 4]   
99988     1645              356         10     [10, 10]     [10, 2, 8]   
99990     1645              347         10      [2, 10]    [10, 4, 10]   
99992     1645              336          6       [3, 9]  [6, 5, 11, 8]   
99995     1645              320          5      [4, 10]    [5, 10, 10]   

      dealer_final_value player_final player_final_value  \
1                     18   [5, 5, 11]              

In [ ]:
df['card1'] = df['player_final'].apply(lambda x: x[0]) #Thanks gemini
df['card2'] = df['player_final'].apply(lambda x: x[1])
df['card3'] = df['player_final'].apply(lambda x: x[2])
#df['card3'] = df['card3'].apply(lambda x: 1 if x > 0 else 0)

for i in range (10000):
  if(df['card1'].iloc[i] == 6 and df['card2'].iloc[i] == 10):
    print(df['card1'].iloc[i])
    print(df['card2'].iloc[i])
    print(df['card3'].iloc[i])


#out = df[['card1', 'card2', 'card3']].values.tolist()

#print(out)
#print(df['card2'])
#print(df['dealer_up'])

#print(df['target'])

6
10
10
6
10
9
6
10
10
6
10
4
6
10
3
6
10
10
6
10
2
6
10
10
6
10
9
6
10
2
6
10
4
6
10
9
6
10
10
6
10
3
6
10
7
6
10
8
6
10
5
6
10
11
6
10
10
6
10
4
6
10
6
6
10
10
6
10
9
6
10
8
6
10
5
6
10
4
6
10
6
6
10
10
6
10
2
6
10
9
6
10
10
6
10
11
6
10
9
6
10
9
6
10
3
6
10
8
6
10
10
6
10
11
6
10
11
6
10
8
6
10
4
6
10
9
6
10
2
6
10
10
6
10
11
6
10
10
6
10
9
6
10
8
6
10
8
6
10
9
6
10
10
6
10
10
6
10
10
6
10
8
6
10
8
6
10
11
6
10
8
6
10
4
6
10
5
6
10
4
6
10
10
6
10
6
6
10
4
6
10
6
6
10
3
6
10
10
6
10
2
6
10
10
6
10
8
6
10
9
6
10
3
6
10
2
6
10
10
6
10
3
6
10
5
6
10
10
6
10
2
6
10
3
6
10
7
6
10
10
6
10
10
6
10
10
6
10
6
6
10
8
6
10
9
6
10
5
6
10
6
6
10
11
6
10
8
6
10
8
6
10
3
6
10
7
6
10
3
6
10
10
6
10
7
6
10
3
6
10
8
6
10
11
6
10
3
6
10
10
6
10
4


In [ ]:
temp = df[df['card1'] == 10]
temp = temp[temp['card2'] == 10]


print(temp)

Empty DataFrame
Columns: [shoe_id, cards_remaining, dealer_up, initial_hand, dealer_final, dealer_final_value, player_final, player_final_value, actions_taken, run_count, true_count, win, target, card1, card2, card3]
Index: []


In [ ]:
# Adapted from Lecture 8 Code

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score

df['card1'] = df['initial_hand'].apply(lambda x: x[0]) #Thanks gemini
df['card2'] = df['initial_hand'].apply(lambda x: x[1])
df['target'] = df['win'].apply(lambda x: 0 if x < 0 else 1) # thanks genimi again

madeUpData = {
    #Uses the pandas dataframe to assign the 3 columns as a nested list
    #card1, card2, dealcard1

    'data': df[['card1', 'card2', 'dealer_up']].values.tolist(),

    #Uses the pandas dataframe to assign the output index column
    'target': df['target'].values.tolist()
}

X, y = madeUpData['data'], madeUpData['target'] ## Dataset Size X 3 ## Dataset Size X 1



                                                     ## 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train) # uses mean and standard deviation of the data to move the data
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.long) # Long used because other functions will be expecting it

class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

input_size = X_train.shape[1] # 3
hidden_size = 8 # just left this number for now
num_classes = 2 # win/loss
model = MLP(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()

#Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr = 0.001)

batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

#30 epochs was randomly assigned for now but seems to give similar results to 100
num_epochs = 30
for epoch in range(num_epochs):
  for batch_X, batch_y in train_loader: # Process each batch separately

    # Forward pass
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
  outputs = model(X_test)
  _, predicted = torch.max(outputs, 1)

accuracy = accuracy_score(y_test, predicted)
print(f'Accuracy: {accuracy * 100:.2f}%')

ValueError: could not convert string to float: '['

In [ ]:
#User input
card1 = int(input("What is your first card: "))
card2 = int(input("What is your second card: "))
dealerCard = int(input("What is the dealers face up card: "))

user_input = torch.tensor([[card1, card2, dealerCard]], dtype = torch.float32)
user_input = scaler.transform(user_input)
user_input = torch.tensor(user_input, dtype = torch.float32)

with torch.no_grad():
  #puts the users input into the model
  outputs = model(user_input)
  _, predicted = torch.max(outputs, 1)

#output is based on what the model predicted
if predicted == 0:
  print("You likely lose")
else:
  print("You likely win")